In [2]:
%reload_ext autoreload
%autoreload 2
import numpy
import os
import sklearn.preprocessing
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from classifier import classification_data

In [3]:
hyperparams = {
    "DecisionTreeClassifier": {
        "criterion": ["gini", "entropy"],
        "splitter": ["best", "random"],
        "max_depth": [10, 100, 1000]
    },
    "KNeighborsClassifier": {
        "n_neighbors": [2, 4, 6, 8, 10],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"]
    },
    "MLPClassifier": {
        "activation": ["identity", "logistic", "tanh", "relu"],
        "solver": ["adam", "sgd"],
        "learning_rate_init": [0.01, 0.001, 0.0001],
        "momentum": [0.9, 0.4, 0.1]
    },
    "RandomForestClassifier": {
        "n_estimators": [200, 400, 600, 800, 1000],
        "max_features": ["sqrt", "log2"],
        "criterion": ["gini", "entropy"],
        "max_depth": [10, 100, 1000]
    },
    "SVC": {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    }
}
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "../dataset",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
}
index = sklearn.model_selection.StratifiedShuffleSplit(n_splits=cfg["fold"], train_size=cfg["train_size"], test_size=cfg["test_size"], random_state=1)

In [4]:
surf = numpy.loadtxt(os.path.join(cfg["path_base"], "surf64.txt"))
n_samples, n_features = surf.shape
x_surf, y_surf = surf[0:, 0:n_features - 1], surf[:, n_features - 1]

In [5]:
mobilenet = numpy.load(os.path.join(cfg["path_base"], "cnn", "horizontal+patch=3+resnet50v2.npz"))
dataset = "resnet50v2"
orientation = "horizontal"
n_patch = 3
x = mobilenet["features"]
y = mobilenet["labels"]
n_samples = x.shape[0]
n_features = x.shape[1]
x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

print(x.shape, y.shape)

(1125, 2048) (1125,)


In [6]:
for pca in (128, 256, 512, 1024, -1):
    x = x_normalized
    if pca > 0:
        x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

    if pca == -1:
        pca = n_features

    classification_data(cfg, dataset, index, pca, n_samples, x, y, n_patch=n_patch, orientation=orientation, x_surf=x_surf, y_surf=y_surf)
    break

0 DecisionTreeClassifier (900, 128) (225, 128)
1 DecisionTreeClassifier (900, 128) (225, 128)
2 DecisionTreeClassifier (900, 128) (225, 128)
3 DecisionTreeClassifier (900, 128) (225, 128)
4 DecisionTreeClassifier (900, 128) (225, 128)
best_accuracy: 50.1333, best_rule: sum

0 KNeighborsClassifier (900, 128) (225, 128)
1 KNeighborsClassifier (900, 128) (225, 128)
2 KNeighborsClassifier (900, 128) (225, 128)
3 KNeighborsClassifier (900, 128) (225, 128)
4 KNeighborsClassifier (900, 128) (225, 128)
best_accuracy: 73.3333, best_rule: sum

0 MyEnsembleClassifier
1 MyEnsembleClassifier
2 MyEnsembleClassifier
3 MyEnsembleClassifier
4 MyEnsembleClassifier
best_accuracy: 62.6667, best_rule: sum

0 VotingClassifier (900, 128) (225, 128)
1 VotingClassifier (900, 128) (225, 128)
2 VotingClassifier (900, 128) (225, 128)
3 VotingClassifier (900, 128) (225, 128)
4 VotingClassifier (900, 128) (225, 128)
best_accuracy: 61.3333

